## Installing NLTK and Streamlit packages

In [1]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importing necessary libraries use to implement chatbot

In [2]:
# Importing necessary libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

 ### Adding intents
 Intents represent the purpose or goal behind a user's input or query. Essentially, an intent describes what the user wants to achieve or the kind of response they're expecting when they interact with the chatbot.

In [4]:
intents = [
    {
        'tag':'greeting',
        'patterns':['Hi', 'Hello', 'Hey', 'Whats up', 'How are you'],
        'responses':['Hi there', 'Hello', 'Hey', 'Nothing much', 'Im fine, thank you']
    },
    {
        'tag':'goodbye',
        'patterns':['Bye', 'See you later', 'Goodbye', 'Take care'],
        'responses':['Goodbye', 'See you later', 'Take care']
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        'tag': 'news',
        'patterns': ['What\'s the news', 'Tell me the news', 'What\'s happening today', 'Any news updates'],
        'responses': ['I don\'t have access to real-time news, but you can check a news app or website for the latest updates.', 'You can always visit your preferred news source for up-to-date information.', 'Sorry, I can\'t provide live news updates. Please check a news website or app for current events.']
    },
    {
        'tag': 'time',
        'patterns': ['What time is it', 'Tell me the time', 'What\'s the current time'],
        'responses': ['I\'m unable to provide real-time information. Please check your device for the current time.', 'You can check the time on your device or watch.']
    },
    {
        'tag': 'joke',
        'patterns': ['Tell me a joke', 'Make me laugh', 'I need a joke', 'Can you tell me something funny'],
        'responses': ['Why don’t skeletons fight each other? They don’t have the guts.', 'Why did the scarecrow win an award? Because he was outstanding in his field!', 'What do you call fake spaghetti? An impasta!']
    },
    {
        'tag': 'location',
        'patterns': ['Where am I', 'Can you track my location', 'What is my location'],
        'responses': ['I don\'t have access to your location. You can check your location on your device.', 'Sorry, I cannot track your location. Please use a map or GPS service for that.']
    },
    {
        'tag': 'exercise',
        'patterns': ['What exercises can I do', 'How can I stay fit', 'Give me some workout ideas'],
        'responses': ['You can start with basic exercises like push-ups, squats, and lunges. For a good full-body workout, try combining cardio and strength training exercises.', 'Staying active is key to fitness! Try activities like walking, jogging, yoga, or home workouts like burpees and planks.', 'A good exercise routine can include aerobic exercises like running or cycling, as well as strength training exercises such as weightlifting.']
    },
    {
        'tag': 'nutrition',
        'patterns': ['What should I eat for a healthy diet', 'Give me some nutrition tips', 'How can I eat healthy'],
        'responses': ['A balanced diet should include a variety of fruits, vegetables, lean proteins, and whole grains. Drink plenty of water and limit processed foods.', 'Eat a mix of vegetables, fruits, healthy fats, and proteins. Consider reducing sugar intake and focusing on whole foods.', 'For a healthy diet, include fiber-rich foods like vegetables, fruits, and whole grains, and avoid excessive processed foods and sugary drinks.']
    },
    {
        'tag': 'exercise_duration',
        'patterns': ['How long should I exercise each day', 'What is the ideal workout time', 'How much should I exercise'],
        'responses': ['It’s generally recommended to exercise for at least 30 minutes a day for good health. This can include any form of physical activity like walking, running, or weight training.', 'The American Heart Association suggests 150 minutes of moderate aerobic activity per week or 75 minutes of vigorous activity, plus strength training twice a week.']
    },
    {
        'tag': 'motivation',
        'patterns': ['I need some motivation', 'Give me a motivational quote', 'How can I stay motivated'],
        'responses': ['“The only way to do great work is to love what you do.” – Steve Jobs', '“Success is not the key to happiness. Happiness is the key to success.” – Albert Schweitzer', '“Don’t watch the clock; do what it does. Keep going.” – Sam Levenson']
    },
    {
        'tag': 'fitness_goal',
        'patterns': ['How do I set fitness goals', 'What are good fitness goals', 'Help me with setting fitness goals'],
        'responses': ['Start by defining clear, realistic goals. For example, you can set goals like running 5 miles in 30 minutes or doing 10 push-ups in a row.', 'A good fitness goal could be improving your strength, increasing your endurance, or working on flexibility. Break them down into small, achievable milestones.', 'To set fitness goals, start by focusing on what you want to achieve (e.g., lose weight, gain muscle). Then, make sure your goals are specific, measurable, and time-bound.']
    },
    {
        'tag': 'sleep',
        'patterns': ['How can I improve my sleep', 'What are tips for better sleep', 'Why am I not sleeping well'],
        'responses': ['Try creating a bedtime routine, avoiding screens before bed, and making your sleeping environment as comfortable as possible.', 'Make sure to go to bed and wake up at the same time every day, limit caffeine intake before bed, and avoid using electronic devices in the hour before sleep.', 'To improve sleep, try to create a calming bedtime routine and avoid stimulants like caffeine or heavy meals close to bedtime.']
    },
    {
        'tag': 'travel',
        'patterns': ['Where should I travel', 'What are the best places to visit', 'Give me some travel recommendations'],
        'responses': ['It depends on what you like! If you enjoy beaches, try places like Bali or Hawaii. For history, consider places like Rome or Athens. If you love nature, visit places like New Zealand or the Swiss Alps.', 'The best travel destination varies by preference, but consider trying destinations like Paris, Tokyo, or Cape Town depending on your interests.']
    }
]
